# Genre-Driven Storytelling from Images using PyTorch XPU backend
## Overview
This sample explores the generation of creative, genre-specific stories from images, specifically optimized for Intel hardware using the PyTorch XPU backend. 

## Workflow
It takes an image and a user-defined genre (e.g., fantasy, horror, romance, sci-fi) as input and leverages a Vision Language Model (VLM) to craft engaging narratives that are visually inspired and thematically aligned with the chosen genre.

<img width="600" alt="image" src="./assets/story-generation.png">

## Import Necessary Packages

In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import torch
import ipywidgets as widgets
from IPython.display import display, Image as IPImage 
from PIL import Image as PILImage 
from qwen_vl_utils import process_vision_info
import io
import os

In [2]:
from huggingface_hub import login
login()

## Story Generation Module

Using Qwen VL Model, users could generate a creative genre-specific story with minimal prompt changes.

In [3]:
def story_generation(image, genre):
    """
        Generates a creative story using Qwen 2.5 VL 3B Instruct model
        Args:
            image(PIL image): User uploaded image
            genre(str): User selected genre(eg. Fantasy, Horror, Sci-fi, etc.)
        Returns: 
            story(str): Model generated story
    """
    try:
        model_id = "Qwen/Qwen2.5-VL-3B-Instruct"
        try:
            min_pixels = 256*28*28
            max_pixels = 1280*28*28
            processor = AutoProcessor.from_pretrained(model_id,
                                                      local_files_only=True,
                                                      use_fast=True,
                                                      min_pixels=min_pixels, 
                                                      max_pixels=max_pixels)
            model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
                model_id,
                local_files_only=True,
                torch_dtype=torch.bfloat16,
            )
        except Exception as e:
            print(f"Model '{model_id}' is not available locally. Please download it first using the CLI command( refer README)")
            raise
        device = "xpu" if torch.xpu.is_available() else "cpu"
        model = model.to(device)
        model.eval()
        compiled_model = torch.compile(model)
            
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": image,
                    },
                    {
                        "type": "text",
                        # "text": prompt,
                        "text": f"Generate a creative {genre} story inspired by this image. Focus on the characters (if any are visible, describe them briefly), the atmosphere of the scene, and the potential narrative that could unfold. Craft an engaging plot and ensure the story conveys a suitable moral."                    
                    },
                ],
            }
        ]
        # Preparation for inference
        text = processor.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = processor(text=[text], 
                           images=[image], 
                           padding=True, 
                           return_tensors="pt")
        inputs = inputs.to("xpu")
    
        torch.xpu.empty_cache()
        # Generation of the output
        with torch.no_grad():
            generated_ids = compiled_model.generate(**inputs, 
                                           temperature=0.9,
                                           top_p=0.99,
                                           top_k=40,
                                           do_sample=True,
                                           max_new_tokens=1024)
            generated_ids_trimmed = [
                out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )
            torch.xpu.synchronize()
        del model, processor, inputs, generated_ids, generated_ids_trimmed
        return output_text[0]
    except Exception as e:
        print("Error generating story: ", e)

## User-provided inputs

### Upload image

Users could also use sample input images from [sample-inputs](./assets/sample-inputs) folder

In [4]:
default_image_path = "./assets/sample-inputs/input1.jpg" 

image = None

uploader = widgets.FileUpload(
    accept='image/*', 
    multiple=False    
)

output_area = widgets.Output()

def _load_and_store_pil_image(source_type, data, filename_for_error_msg=None):
    """
    Loads image data into a PIL.Image object and stores it globally.
    source_type: 'path' (data is a file path) or 'bytes' (data is image byte content).
    Returns PIL.Image object on success, None on failure.
    """
    global image
    try:
        if source_type == 'path':
            pil_img = PILImage.open(data)
        elif source_type == 'bytes':
            pil_img = PILImage.open(io.BytesIO(data))
        else: 
            image = None
            return None 
        
        image = pil_img
        return pil_img
    except Exception as e:
        image = None
        print(f"Error loading image: {e}")

def display_default_image_handler():
    """Loads the default image as a PIL object, stores it, and displays it."""
    with output_area:
        output_area.clear_output() 
        if not os.path.exists(default_image_path):
            global image
            image = None
            print(f"Default image not found: {default_image_path}. Please check the path.")
            return

        pil_img = _load_and_store_pil_image('path', default_image_path)
        if pil_img:
            display(IPImage(filename=default_image_path))
            print(f"Displaying default image: {default_image_path}. Stored as PIL.")
        else:
            print(f"Error loading default PIL image from '{default_image_path}'.")

def on_upload_event_handler(change):
    """Handles file upload/clear, stores as PIL Image, and displays."""
    with output_area:
        output_area.clear_output()

        if not uploader.value: 
            print("No file uploaded. Displaying default image.")
            display_default_image_handler() 
            return

        
        uploaded_file_info = uploader.value[0]
        file_content_bytes = uploaded_file_info['content']
        file_name = uploaded_file_info['name']

        pil_img = _load_and_store_pil_image('bytes', file_content_bytes, filename_for_error_msg=file_name)
        if pil_img:
            display(IPImage(data=file_content_bytes)) 
            print(f"Displayed uploaded image: {file_name}. Stored as PIL.")
        else:
            print(f"Error processing uploaded image '{file_name}' into PIL format.")

uploader.observe(on_upload_event_handler, names='value')

print("Please upload an image file (e.g., jpg, png, gif).")
print(f"If no image is uploaded, a default image will be attempted from: {default_image_path}")
display(uploader)
display(output_area)

with output_area:
    output_area.clear_output(wait=True) 
    display_default_image_handler() 


Please upload an image file (e.g., jpg, png, gif).
If no image is uploaded, a default image will be attempted from: ./assets/sample-inputs/input1.jpg


FileUpload(value=(), accept='image/*', description='Upload')

Output()

### Select Genre for the story

Specify the genre in which user would want the VL Model to generate the story.

In [5]:
# Top genres are listed using RadioButtons function from ipywidgets
genre = widgets.RadioButtons(
    options=['Fantasy', 'Horror', 'Science Fiction', 'Thriller and Suspense', 'Romance', 'Historical fiction'],
    value='Fantasy', # Default'
    description='Genre',
    disabled=False
)
display(genre)

RadioButtons(description='Genre', options=('Fantasy', 'Horror', 'Science Fiction', 'Thriller and Suspense', 'R…

In [6]:
torch.xpu.empty_cache()

## Story Generation

This orchestrates the story generation by taking an image and a user-defined genre as input and then producing a narrative aligned with the chosen genre.

In [7]:
print(genre.value)
story = story_generation(image, genre.value)
print(story)

Science Fiction


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In the vast wilderness of the American West, under a clear blue sky dotted with wispy clouds, stood an ancient white stallion named Luna. Luna was no ordinary horse; he belonged to a special breed known as the Freedom Runners, created by a secretive organization dedicated to preserving endangered equine species. Luna's coat shimmered in the sun, and his mane flowed like a waterfall.

Near Luna sat a peculiar looking dog, sleek with a golden coat and expressive green eyes. This dog was named Shadow, a modern mutt who had been specially trained by the organization to accompany Luna during his daily patrols. Shadow had once belonged to one of the founder's children, but tragedy had separated him from the family years ago. He now lived with the Freedom Runners as part of their conservation efforts.

Their world was one of harmony and balance, where humans, animals, and nature coexisted peacefully, respecting each other's roles within the ecosystem. The organization was a secret, dedicated 

## Sample Outputs

### Fantasy Genre

In [ ]:
print(genre.value)
story = story_generation(image, genre.value)
print(story)

### Science Fiction

In [ ]:
print(genre.value)
story = story_generation(image, genre.value)
print(story)